In [215]:
%cd '/content/drive/MyDrive/Colab Notebooks/NLP_HW2'

/content/drive/MyDrive/Colab Notebooks/NLP_HW2


In [ ]:
!pip install rouge-score transformers

     |████████████████████████████████| 4.2 MB 7.1 MB/s 
     |████████████████████████████████| 84 kB 3.0 MB/s 
     |████████████████████████████████| 6.6 MB 45.3 MB/s 
     |████████████████████████████████| 596 kB 50.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!python main.py

# multihead attention

In [248]:
import torch
from torch import nn
import torch.nn.functional as F

In [217]:
a=[1,2,3]
a= torch.tensor(a)
a.shape[0]

3

In [259]:
query = torch.tensor([[[1,2,3,4],[1,2,3,4]],[[11,12,13,14],[11,12,13,14]],[[21,22,23,24],[21,22,23,24]]], dtype=torch.float32)
key = torch.tensor([[[1,2,3,4],[1,2,3,4]],[[11,12,13,14],[11,12,13,14]],[[21,22,23,24],[21,22,23,24]]], dtype=torch.float32)
value = torch.tensor([[[1,2,3,4],[1,2,3,4]],[[11,12,13,14],[11,12,13,14]],[[21,22,23,24],[21,22,23,24]]], dtype=torch.float32)

In [219]:
query.shape
#batch = 3
#target_len = 2
#hidden = 4

torch.Size([3, 2, 4])

In [220]:
#num head = 2
#d_feat=4
#d_head = 4/2 = 2
def _shape(tensor, seq_len, batch_size):
        return tensor.view(batch_size, seq_len, 2, 2).transpose(1, 2).contiguous()

query_split = _shape(query, 2, 3)
query_split

tensor([[[[ 1.,  2.],
          [ 1.,  2.]],

         [[ 3.,  4.],
          [ 3.,  4.]]],


        [[[11., 12.],
          [11., 12.]],

         [[13., 14.],
          [13., 14.]]],


        [[[21., 22.],
          [21., 22.]],

         [[23., 24.],
          [23., 24.]]]])

In [221]:
key_split = key.view(3, -1, 2, 2).permute(0, 2, 1, 3)
key_split

tensor([[[[ 1.,  2.],
          [ 1.,  2.]],

         [[ 3.,  4.],
          [ 3.,  4.]]],


        [[[11., 12.],
          [11., 12.]],

         [[13., 14.],
          [13., 14.]]],


        [[[21., 22.],
          [21., 22.]],

         [[23., 24.],
          [23., 24.]]]])

In [222]:
value_split = value.view(3, -1, 2, 2).permute(0, 2, 1, 3)
value_split

tensor([[[[ 1.,  2.],
          [ 1.,  2.]],

         [[ 3.,  4.],
          [ 3.,  4.]]],


        [[[11., 12.],
          [11., 12.]],

         [[13., 14.],
          [13., 14.]]],


        [[[21., 22.],
          [21., 22.]],

         [[23., 24.],
          [23., 24.]]]])

In [223]:
key_split.shape

torch.Size([3, 2, 2, 2])

In [224]:
matmul_qk = torch.matmul(query_split, key_split.permute(0, 1, 3, 2))
matmul_qk

tensor([[[[   5.,    5.],
          [   5.,    5.]],

         [[  25.,   25.],
          [  25.,   25.]]],


        [[[ 265.,  265.],
          [ 265.,  265.]],

         [[ 365.,  365.],
          [ 365.,  365.]]],


        [[[ 925.,  925.],
          [ 925.,  925.]],

         [[1105., 1105.],
          [1105., 1105.]]]])

In [225]:
depth = torch.tensor(key.shape[-1], dtype=torch.float32) 
logits = matmul_qk / torch.sqrt(depth)
logits

tensor([[[[  2.5000,   2.5000],
          [  2.5000,   2.5000]],

         [[ 12.5000,  12.5000],
          [ 12.5000,  12.5000]]],


        [[[132.5000, 132.5000],
          [132.5000, 132.5000]],

         [[182.5000, 182.5000],
          [182.5000, 182.5000]]],


        [[[462.5000, 462.5000],
          [462.5000, 462.5000]],

         [[552.5000, 552.5000],
          [552.5000, 552.5000]]]])

In [226]:
d_K = torch.tensor(key.size()[-1])
scores = torch.matmul(query_split, key_split.permute(0, 1, 3, 2)) / torch.sqrt(d_K)
scores.dtype

torch.float32

In [227]:
# 소프트맥스 함수는 마지막 차원인 key의 문장 길이 방향으로 수행된다.
# attention weight : (batch_size, num_heads, query의 문장 길이, key의 문장 길이)
attention = torch.softmax(scores, dim=-1)
attention

tensor([[[[0.5000, 0.5000],
          [0.5000, 0.5000]],

         [[0.5000, 0.5000],
          [0.5000, 0.5000]]],


        [[[0.5000, 0.5000],
          [0.5000, 0.5000]],

         [[0.5000, 0.5000],
          [0.5000, 0.5000]]],


        [[[0.5000, 0.5000],
          [0.5000, 0.5000]],

         [[0.5000, 0.5000],
          [0.5000, 0.5000]]]])

In [228]:
dropout = nn.Dropout(0.1)
out_proj = nn.Linear(4,4)

In [229]:
x_raw = torch.matmul(dropout(attention), value_split)
x_rsh1 = x_raw.permute(0,2,1,3).contiguous()
x_rsh1

tensor([[[[ 1.1111,  2.2222],
          [ 1.6667,  2.2222]],

         [[ 1.1111,  2.2222],
          [ 1.6667,  2.2222]]],


        [[[12.2222, 13.3333],
          [ 7.2222,  7.7778]],

         [[12.2222, 13.3333],
          [14.4444, 15.5556]]],


        [[[23.3333, 24.4444],
          [12.7778, 13.3333]],

         [[23.3333, 24.4444],
          [25.5556, 26.6667]]]])

In [230]:
x_rsh2 = x_rsh1.view(3,-1,4)
x_rsh2

tensor([[[ 1.1111,  2.2222,  1.6667,  2.2222],
         [ 1.1111,  2.2222,  1.6667,  2.2222]],

        [[12.2222, 13.3333,  7.2222,  7.7778],
         [12.2222, 13.3333, 14.4444, 15.5556]],

        [[23.3333, 24.4444, 12.7778, 13.3333],
         [23.3333, 24.4444, 25.5556, 26.6667]]])

In [231]:
x = out_proj(x_rsh2)
x

tensor([[[ -1.1701,   0.0275,  -1.1662,  -1.0320],
         [ -1.1701,   0.0275,  -1.1662,  -1.0320]],

        [[ -7.1726,   5.2987,  -3.7255,  -6.2647],
         [ -7.9328,   4.0345,  -5.8744, -11.0123]],

        [[-13.1751,  10.5698,  -6.2848, -11.4975],
         [-14.3899,   8.4228, -10.0431, -19.7106]]], grad_fn=<AddBackward0>)

In [232]:
x.shape

torch.Size([3, 2, 4])

# encoder

In [233]:
query.shape

torch.Size([3, 2, 4])

In [234]:
enc_mask = [[[[1,1],[1,1]]],[[[1,1],[1,1]]],[[[1,1],[1,1]]]]
enc_mask = torch.tensor(enc_mask)
enc_mask.shape

torch.Size([3, 1, 2, 2])

In [235]:
if enc_mask is not None:
  scores = scores.masked_fill(enc_mask==0,-1e9)
scores

tensor([[[[  2.5000,   2.5000],
          [  2.5000,   2.5000]],

         [[ 12.5000,  12.5000],
          [ 12.5000,  12.5000]]],


        [[[132.5000, 132.5000],
          [132.5000, 132.5000]],

         [[182.5000, 182.5000],
          [182.5000, 182.5000]]],


        [[[462.5000, 462.5000],
          [462.5000, 462.5000]],

         [[552.5000, 552.5000],
          [552.5000, 552.5000]]]])

In [236]:
attention = x
attention = dropout(attention)
attention

tensor([[[ -1.3001,   0.0306,  -1.2958,  -1.1466],
         [ -1.3001,   0.0306,  -1.2958,  -1.1466]],

        [[ -7.9696,   5.8874,  -4.1394,  -6.9608],
         [ -0.0000,   4.4828,  -6.5271,  -0.0000]],

        [[-14.6390,  11.7442,  -6.9831, -12.7750],
         [-15.9888,   9.3586, -11.1591, -21.9007]]], grad_fn=<MulBackward0>)

In [237]:
#d_model = feature 개수 = 쭉 4로 쓰고있다
self_attn_layer_norm = nn.LayerNorm(4)

In [238]:
attention = self_attn_layer_norm(query + attention)

In [239]:
attention

tensor([[[-1.6151,  0.3956,  0.1141,  1.1054],
         [-1.6151,  0.3956,  0.1141,  1.1054]],

        [[-1.1352,  1.5965, -0.0632, -0.3981],
         [-0.2651,  1.2049, -1.4789,  0.5392]],

        [[-1.0130,  1.6349, -0.0793, -0.5427],
         [-0.6634,  1.6467, -0.0646, -0.9187]]],
       grad_fn=<NativeLayerNormBackward0>)

In [240]:
activation_fn = nn.ReLU()
fc1 = nn.Linear(4, 4 * 4)
fc2 = nn.Linear(4 * 4, 4)

In [241]:
outputs = activation_fn(fc1(attention))
outputs = fc2(outputs)
outputs

tensor([[[-0.3388,  0.4176, -0.1812, -0.1993],
         [-0.3388,  0.4176, -0.1812, -0.1993]],

        [[-0.2993,  0.6982, -0.0226, -0.1298],
         [-0.1862,  0.5623, -0.2182, -0.1279]],

        [[-0.3240,  0.6769, -0.0051, -0.0826],
         [-0.3769,  0.5859,  0.0335,  0.0445]]], grad_fn=<AddBackward0>)

# decoder loss

In [242]:
vocab_size = 16
lm_head = nn.Linear(4, 16, bias = False)

In [243]:
out = lm_head(outputs)
out.shape

torch.Size([3, 2, 16])

In [244]:
out.shape

torch.Size([3, 2, 16])

In [245]:
label = torch.tensor([[1,2],[3,4],[5,6]])
label.shape

torch.Size([3, 2])

In [246]:
criterion = nn.CrossEntropyLoss()

for i in range(out.shape[0]):
  lm_loss += criterion(out[i], label[i])

lm_loss

tensor(10.9883, grad_fn=<AddBackward0>)

In [247]:
out2 = out.view[1, -1, ]
out2.shape

TypeError: ignored

In [ ]:
a = torch.tensor([[ 0.0369, -0.1205, -0.1654,  0.1190, -0.0155, -0.0353, -0.2014,
          -0.0700, -0.0117, -0.0997,  0.0747,  0.1583,  0.0943,  0.0483,
           0.1212, -0.3022]])

b = torch.tensor([1])
lm_loss = criterion(a,b)
lm_loss

tensor(2.8778)

In [ ]:
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)

output

tensor(1.8135, grad_fn=<NllLossBackward0>)

In [ ]:
target

tensor([1, 3, 0])

In [ ]:
target

tensor([2, 3, 0])

# masking

In [249]:
def _expand_mask(mask, tgt_len = None):
    """
        Inputs
            mask.shape = (B, S_L)
        Outputs
            output.shape = (B, 1, T_L, S_L)
    """
    batch_size, src_len = mask.size()
    tgt_len = tgt_len if tgt_len is not None else src_len

    expanded_mask = mask[:, None, None, :].expand(batch_size, 1, tgt_len, src_len).to(torch.float)

    inverted_mask = 1.0 - expanded_mask

    return inverted_mask.masked_fill(inverted_mask.bool(), torch.finfo(torch.float).min)

In [267]:
# batch size 2, sequence length 3
mask = torch.tensor([[1,2,3],[1,2,3]])
mask.shape

torch.Size([2, 3])

In [268]:
expand_mask = _expand_mask(mask, 3)
expand_mask

tensor([[[[ 0.0000e+00, -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00, -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00, -3.4028e+38, -3.4028e+38]]],


        [[[ 0.0000e+00, -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00, -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00, -3.4028e+38, -3.4028e+38]]]])

In [269]:
expand_mask.shape
#

torch.Size([2, 1, 3, 3])

In [254]:
def _make_causal_mask(dec_ids):
    """
        Inputs
            dec_ids.shape = (B, D_L)
        Outputs
            output.shape = (B, 1, D_L, D_L)
    """
    batch_size, tgt_len = dec_ids.size()
    device = dec_ids.device

    mask = torch.full((tgt_len, tgt_len), float("-inf"))
    mask_cond = torch.arange(mask.size(-1))
    mask.masked_fill_(mask_cond < (mask_cond + 1).view(mask.size(-1), 1), 0)
    mask = mask.to(torch.float).to(device)

    return mask[None, None, :, :].expand(batch_size, 1, tgt_len, tgt_len)


In [264]:
dec_ids = torch.tensor([[1,2,0],[1,2,3]])
dec_ids.shape

torch.Size([2, 3])

In [265]:
make_causal_mask = _make_causal_mask(dec_ids)
make_causal_mask

tensor([[[[0., -inf, -inf],
          [0., 0., -inf],
          [0., 0., 0.]]],


        [[[0., -inf, -inf],
          [0., 0., -inf],
          [0., 0., 0.]]]])

In [263]:
#batch 2 target len 3 hidden 4
#                         난       점심         먹는다.        그리고       저녁도       먹는다
query = torch.tensor([[[1,2,3,4],[5,6,7,8],[9,10,11,12]],[[1,2,3,4],[5,6,7,8],[9,10,11,12]]])
query.shape

torch.Size([2, 3, 4])

In [266]:
make_causal_mask.shape 
# 난/점심/먹는다 
#[[o x x],[o x x]
 #[o o x],[o o x]
 #[o o o],[o o o]]

torch.Size([2, 1, 3, 3])